In [8]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [9]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "lJrKhZKFXCGXSoYzh7f7",
           "X-Naver-Client-Secret":"YYPQvTkBxR"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [15]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"",
    "ages":["20"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='20'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='20'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='20'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["디지털/가전"], axis=1, inplace=True)
df_all_a

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,6.04648,0.10834,1.05858,1.49650,4.13670,2.30338,0.88026,0.30601,0.92371,0.63326,...,4.75753,0.88026,1.17055,0.44029,1.62722,0.03944,0.07397,PC+MO,전체,20
2021-05-11,5.69324,0.09456,0.91425,1.36849,3.71780,2.13457,0.72469,0.29887,0.91436,0.62764,...,4.20387,0.72469,1.09446,0.39684,1.42468,0.03479,0.05949,PC+MO,전체,20
2021-05-12,5.48206,0.08375,0.86146,1.28997,3.52830,2.45366,0.75878,0.28596,0.92770,0.58597,...,4.01874,0.75878,1.04437,0.38025,1.32218,0.03144,0.06187,PC+MO,전체,20
2021-05-13,5.01507,0.07943,0.79634,1.23031,3.30631,3.78778,0.64709,0.28693,0.85919,0.54058,...,3.74444,0.64709,0.92360,0.38820,1.26328,0.03782,0.05587,PC+MO,전체,20
2021-05-14,4.58375,0.06619,0.72355,1.12440,2.98074,4.77223,0.63293,0.24867,0.77359,0.48833,...,3.39126,0.63293,0.81909,0.32157,1.14618,0.02745,0.04938,PC+MO,전체,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,3.33621,0.05339,0.51016,0.78659,1.46020,2.48462,0.27436,0.09540,0.45252,0.31476,...,1.73601,0.27436,0.50165,0.22482,0.80608,0.03224,0.05384,MO,전체,20
2023-06-30,2.98444,0.05166,0.44861,0.71705,1.29705,2.12066,0.25241,0.08528,0.39798,0.30074,...,1.58911,0.25241,0.45246,0.20649,0.85090,0.02816,0.04609,MO,전체,20
2023-07-01,3.48052,0.07155,0.54464,0.81970,1.53411,2.46358,0.30327,0.09815,0.40350,0.28907,...,1.79060,0.30327,0.49401,0.22011,0.90412,0.03712,0.05074,MO,전체,20


In [16]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"f",
    "ages":["20"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='20'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='20'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='20'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["디지털/가전"], axis=1, inplace=True)
df_all_b

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,7.35066,0.10481,0.88828,2.11858,5.43506,2.47539,0.82359,0.15287,0.22636,0.40680,...,4.49678,0.82359,1.60878,0.21037,1.20508,0.04430,0.09301,PC+MO,여성,20
2021-05-11,6.86093,0.10534,0.78154,1.96130,4.94210,2.32284,0.71363,0.15480,0.24063,0.38878,...,4.00522,0.71363,1.51791,0.18892,1.10595,0.03497,0.07670,PC+MO,여성,20
2021-05-12,6.64894,0.09011,0.75332,1.85628,4.65030,2.71109,0.75332,0.15169,0.19964,0.36732,...,3.87734,0.75332,1.42168,0.19235,0.98902,0.03293,0.07799,PC+MO,여성,20
2021-05-13,6.08389,0.08786,0.70504,1.80746,4.37416,4.19124,0.66717,0.13603,0.22732,0.37365,...,3.59937,0.66717,1.27492,0.21123,0.97014,0.03947,0.07895,PC+MO,여성,20
2021-05-14,5.55714,0.07338,0.64883,1.60974,3.84548,5.39526,0.61568,0.12551,0.19085,0.30746,...,3.18709,0.61568,1.13084,0.15105,0.88099,0.03111,0.06619,PC+MO,여성,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,3.91656,0.04687,0.46916,1.11599,1.82788,2.80046,0.23223,0.06606,0.10724,0.25299,...,1.59720,0.23223,0.63388,0.09363,0.68544,0.03281,0.06550,MO,여성,20
2023-06-30,3.47753,0.04530,0.40354,0.99758,1.62923,2.34279,0.23402,0.05423,0.10311,0.24652,...,1.46462,0.23402,0.57451,0.08715,0.76869,0.03292,0.05658,MO,여성,20
2023-07-01,4.21978,0.07890,0.50431,1.15505,1.93356,2.79856,0.27419,0.06495,0.09474,0.23391,...,1.69106,0.27419,0.65318,0.10188,0.80686,0.04754,0.06059,MO,여성,20


In [17]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"m",
    "ages":["20"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='20'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='20'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='20'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["디지털/가전"], axis=1, inplace=True)
df_all_c

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,4.72405,0.11195,1.23173,0.86537,2.81970,2.12934,0.93801,0.46154,1.63186,0.86330,...,5.02344,0.93801,0.72597,0.67380,2.05616,0.03452,0.05467,PC+MO,남성,20
2021-05-11,4.50929,0.08364,1.04920,0.76703,2.47589,1.94398,0.73610,0.44521,1.59853,0.87027,...,4.40659,0.73610,0.66487,0.60802,1.74861,0.03463,0.04203,PC+MO,남성,20
2021-05-12,4.29888,0.07732,0.97145,0.71540,2.39018,2.19295,0.76452,0.42234,1.66703,0.80808,...,4.16329,0.76452,0.66160,0.57110,1.66072,0.02994,0.04552,PC+MO,남성,20
2021-05-13,3.93132,0.07089,0.88922,0.64472,2.22311,3.37916,0.62686,0.44020,1.50084,0.71018,...,3.89266,0.62686,0.56718,0.56794,1.56118,0.03615,0.03245,PC+MO,남성,20
2021-05-14,3.59676,0.05891,0.79959,0.63198,2.10364,4.14096,0.65061,0.37376,1.36536,0.67206,...,3.59937,0.65061,0.50282,0.49476,1.41568,0.02374,0.03234,PC+MO,남성,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,2.68617,0.05966,0.54773,0.43193,1.05796,2.12531,0.31436,0.12518,0.81017,0.37613,...,1.85630,0.31436,0.35644,0.36019,0.92364,0.03129,0.04102,MO,남성,20
2023-06-30,2.43123,0.05778,0.49100,0.41423,0.93301,1.86392,0.26806,0.11674,0.70316,0.35445,...,1.70017,0.26806,0.31893,0.32960,0.92786,0.02285,0.03457,MO,남성,20
2023-07-01,2.66027,0.06306,0.58079,0.45724,1.09688,2.08417,0.32854,0.13127,0.72191,0.34390,...,1.87283,0.32854,0.32139,0.34191,0.99572,0.02555,0.03985,MO,남성,20


In [18]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7064 entries, 2021-05-10 to 2023-07-03
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   휴대폰액세서리    7064 non-null   float64
 1   생활가전       7064 non-null   float64
 2   영상가전       7064 non-null   float64
 3   이미용가전      7064 non-null   float64
 4   주방가전       7064 non-null   float64
 5   계절가전       7064 non-null   float64
 6   카메라/캠코더용품  7064 non-null   float64
 7   PC         7064 non-null   float64
 8   PC부품       7064 non-null   float64
 9   PC액세서리     7064 non-null   float64
 10  저장장치       7064 non-null   float64
 11  멀티미디어장비    7064 non-null   float64
 12  모니터        7064 non-null   float64
 13  모니터주변기기    7064 non-null   float64
 14  노트북        7064 non-null   float64
 15  노트북액세서리    7064 non-null   float64
 16  소프트웨어      7064 non-null   float64
 17  주변기기       7064 non-null   float64
 18  음향가전       7064 non-null   float64
 19  태블릿PC      7064 non-null   float64
 20

In [19]:
## 엑셀에 저장
df_all.to_excel("MCLS_04_all_fm_20.xlsx", sheet_name="click ratio")
df_all.head()

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,6.04648,0.10834,1.05858,1.49650,4.13670,2.30338,0.88026,0.30601,0.92371,0.63326,...,4.75753,0.88026,1.17055,0.44029,1.62722,0.03944,0.07397,PC+MO,전체,20
2021-05-11,5.69324,0.09456,0.91425,1.36849,3.71780,2.13457,0.72469,0.29887,0.91436,0.62764,...,4.20387,0.72469,1.09446,0.39684,1.42468,0.03479,0.05949,PC+MO,전체,20
2021-05-12,5.48206,0.08375,0.86146,1.28997,3.52830,2.45366,0.75878,0.28596,0.92770,0.58597,...,4.01874,0.75878,1.04437,0.38025,1.32218,0.03144,0.06187,PC+MO,전체,20
2021-05-13,5.01507,0.07943,0.79634,1.23031,3.30631,3.78778,0.64709,0.28693,0.85919,0.54058,...,3.74444,0.64709,0.92360,0.38820,1.26328,0.03782,0.05587,PC+MO,전체,20
2021-05-14,4.58375,0.06619,0.72355,1.12440,2.98074,4.77223,0.63293,0.24867,0.77359,0.48833,...,3.39126,0.63293,0.81909,0.32157,1.14618,0.02745,0.04938,PC+MO,전체,20
